In [56]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import warnings
from sklearn import tree
from IPython.display import Image
import pydotplus as pydot 
warnings.filterwarnings("ignore")
from sklearn.externals.six import StringIO
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from numpy import set_printoptions
import tensorflow as tf
set_printoptions(precision=3)

In [2]:
Data = pd.read_excel("Gen.xlsx",sheet_name="data").dropna(how = "any")
Data['Generator 1capacity (KVA)'] = Data['Generator 1capacity (KVA)'].astype("category")
Data = Data.dropna(how = 'any')

In [3]:
Data['consumption_perDay_within_a_period'] = Data['Consumption HIS']/(Data['NBRE DE JOUR'])
Data.loc[~np.isfinite(Data['consumption_perDay_within_a_period']), 'consumption_perDay_within_a_period'] = 0
Data['Quanitity_consumed_btn_visits'] = Data['QTE FUEL PRECEDENTE'] - Data['QTE FUEL TROUVEE']
set_printoptions(precision=3)

In [4]:
lb_make = LabelEncoder()
Data["Gen_capacityCode"] = lb_make.fit_transform(Data["Generator 1capacity (KVA)"])
lab_enc = preprocessing.LabelEncoder()
#y = lab_enc.fit_transform(Data['Consumption HIS'])
#Data[["Generator 1capacity (KVA)", "Gen_capacityCode"]]
Data.head(3)
#Data1 = pd.get_dummies(Data['Cluster'],prefix=['Cluster'], drop_first=True)

,Cluster,Site Name,DATE EFFECTIVE DE LA VISITE,DATE DE VISITE PRECEDENTE,Months,NBRE DE JOUR,Generator 1capacity (KVA),COMPTEUR HORAIRE ACTUEL GE1,ANCIEN COMPTEUR HORAIRE G1,QTE FUEL PRECEDENTE,...,QTE FUEL AJOUTEE,QTE TOTALE LAISSEE,Consumption HIS,Class,NBRE HEURES DE FONCTIONNEMENT DG1,CONSOMMATION HORAIRE DE FUEL DG1,Power Type,consumption_perDay_within_a_period,Quanitity_consumed_btn_visits,Gen_capacityCode
0,KOUSSERI,FIGUIL [ T0157 ],2017-09-26,2017-09-20,September,6,22.0,21136,20996,461.0,...,375.0,486.0,420.0,-1,140.0,3.0,Gen Only,70.0,350.0,15
1,GUIDER,DEMBO [ T1408 ],2017-09-26,2017-09-22,September,4,22.0,2522,2424,685.0,...,0.0,440.0,254.8,-1,98.0,2.6,Gen Only,63.7,245.0,15
2,GAROUA 1,BANGANA [ T2901 ],2017-09-26,2017-09-21,September,5,9.0,13284,13165,495.0,...,200.0,480.0,238.0,-1,119.0,2.0,Gen Only,47.6,215.0,4


In [5]:
Data.drop(['Power Type'],axis=1, inplace=True)
Data.drop(['ANCIEN COMPTEUR HORAIRE G1'],axis=1, inplace=True)
Data.drop(['Class'],axis=1, inplace=True)
Data.drop(['DATE  EFFECTIVE DE LA VISITE'],axis=1, inplace=True)
Data.drop(['DATE DE VISITE PRECEDENTE'],axis=1, inplace=True)

Data.drop(['Months'],axis=1, inplace=True)
Data.drop(['Cluster'],axis=1, inplace=True)
Data.drop(['Site Name'],axis=1, inplace=True)
Data.drop(['Generator 1capacity (KVA)'],axis=1, inplace=True)

Data.drop(['COMPTEUR HORAIRE ACTUEL GE1'],axis=1, inplace=True)
Data.drop(['QTE FUEL AJOUTEE'],axis=1, inplace=True)


In [6]:

Data['Running time per day'] = Data["NBRE HEURES DE FONCTIONNEMENT DG1"]/Data["NBRE DE JOUR"]             
Data.loc[~np.isfinite(Data['Running time per day']), 'Running time per day'] = 0
set_printoptions(precision=3)

In [7]:
Data['Maximum_consumption_perDay'] = Data[['CONSOMMATION HORAIRE DE FUEL DG1']]*24
set_printoptions(precision=3)

maximum consumption in one day
fuel consumed per period difference netween the fuel found and previously left
comparison of the difference with running time.
Generator running for more than 24 hourse in a day


In [8]:
a = list(Data['NBRE HEURES DE FONCTIONNEMENT DG1'])
b = list(Data['Quanitity_consumed_btn_visits'])

In [9]:
c = []
for i in range(len(a)):
    if a[i] == 0  and  b[i]!=0:
        c.append(0)
    else:
        c.append(1)

In [10]:
t = list(Data['Maximum_consumption_perDay'])
r = list (Data['consumption_perDay_within_a_period'])

In [11]:
z = []
for i in range(len(r)):
    if r[i] > t[i]:
        z.append(0)
    else:
        z.append(1)

In [12]:
p = []
for i in range(len(z)):
    if c[i]==1 and z[i]==1:
        p.append(1)
    else:
        p.append(0)     

In [13]:
Data['Class'] = p


In [14]:
Data.drop(['QTE TOTALE LAISSEE'],axis=1, inplace=True)
#Data.drop(['Consumption HIS'],axis=1, inplace=True)
Data.drop(['Gen_capacityCode'],axis=1, inplace=True)
Data.drop(['QTE FUEL PRECEDENTE'],axis=1, inplace=True)
Data.drop(['QTE FUEL TROUVEE'],axis=1, inplace=True)
#Data.drop(['NBRE HEURES DE FONCTIONNEMENT DG1'],axis=1, inplace=True)
#Data.drop(['CONSOMMATION HORAIRE DE FUEL DG1'],axis=1, inplace=True)
Data.drop(['NBRE DE JOUR'],axis=1, inplace=True)
Data.drop(['Quanitity_consumed_btn_visits'],axis=1, inplace=True)
Data.drop(['Maximum_consumption_perDay'],axis=1, inplace=True)

In [ ]:
Data.columns

In [38]:
# FUNCTION FOR TRAINING THE MODEL
def trainmodel(model):
    model.fit(x_train,y_train)
# FUNCTION TO MAKE PREDICTIONS

def predictmodel(model):
    y_pred = model.predict(x_test)
    f,t,thresholds = metrics.roc_curve(y_test,y_pred)
    cm = metrics.confusion_matrix(y_test,y_pred)
    
    print("Score:", metrics.auc(f,t))
    print("Classification report:")
    print(metrics.classification_report(y_test,y_pred))
    print("Confusion Matrix:")
    print(cm)
def predictmodeln(model):
    y_pred = model.predict(x_test)
    f,t,thresholds = metrics.roc_curve(y_test,y_pred)
    cm = metrics.confusion_matrix(y_test,y_pred)
    print("Score:", metrics.auc(f,t))
    print("Classification report:")
    print(metrics.classification_report(y_test,y_pred))
    print("Confusion Matrix:")
    print(cm)
    

In [48]:
x = Data.loc[:,Data.columns.tolist()[0:12]]
x = x.as_matrix()
Y = Data.loc[:,'Class']
Y = Y.as_matrix()
x_train,x_test,y_train,y_test = train_test_split(x,Y,test_size=0.33,random_state=7)
scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test) 

clf = MLPClassifier( activation='relu', beta_1=0.6, beta_2=0.9,alpha = 0.001,early_stopping = True,shuffle = True,warm_start = True,validation_fraction = 0.3,learning_rate_init=0.01, max_iter = 14000, random_state = 2700, learning_rate='adaptive')
clf.fit(x_train , y_train)
predictmodel(clf)
print(x.shape)
print(Y.shape)
print(x_test.shape)
print( x_train.shape)
print( y_test.shape)
print(y_train.shape)

Score: 0.8774728675316636
Classification report:
             precision    recall  f1-score   support

          0       0.89      0.81      0.85       322
          1       0.90      0.94      0.92       562

avg / total       0.89      0.89      0.89       884

Confusion Matrix:
[[262  60]
 [ 33 529]]
(2676, 12)
(2676,)
(884, 12)
(1792, 12)
(884,)
(1792,)


In [74]:
y_pred= clf.predict(x_test)
print('Overall AUC:', roc_auc_score(y_test, clf.predict(x_test)))

submission = pd.DataFrame({"Original":y_test,"predict":y_pred})

Overall AUC: 0.8774728675316636


In [75]:
submission

,Original,predict
0,1,1
1,0,0
2,0,0
3,1,1
4,0,0
5,1,1
6,1,1
7,0,1
8,1,1
9,0,0
